In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from textblob import TextBlob
from nba_api.stats.endpoints import leaguedashteamstats
import re

In [13]:
# List of seasons to loop through
seasons = [f"{year}-{str(year+1)[-2:]}" for year in range(2015, 2025)]

# Dictionary to hold data for each season
season_data_frames = {}

# Loop through each season and fetch data
for season in seasons:
    print(f"Fetching data for season: {season}")
    df = leaguedashteamstats.LeagueDashTeamStats(
        season=season,
        per_mode_detailed='PerGame'
    ).get_data_frames()[0]
    season_data_frames[season] = df

df_2015_2016 = season_data_frames['2015-16']
sorted_pts = df_2015_2016.sort_values('REB_RANK')
sorted_pts.tail()


Fetching data for season: 2015-16
Fetching data for season: 2016-17
Fetching data for season: 2017-18
Fetching data for season: 2018-19
Fetching data for season: 2019-20
Fetching data for season: 2020-21
Fetching data for season: 2021-22
Fetching data for season: 2022-23
Fetching data for season: 2023-24
Fetching data for season: 2024-25


,TEAM_ID,TEAM_NAME,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,...,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
29,1610612764,Washington Wizards,82,41,41,0.500,48.2,39.5,85.8,0.460,...,26,5,16,8,26,6,21,16,9,17
16,1610612749,Milwaukee Bucks,82,33,49,0.402,48.4,38.4,82.2,0.467,...,27,9,25,13,7,26,17,21,25,26
14,1610612763,Memphis Grizzlies,82,42,40,0.512,48.4,36.8,83.6,0.440,...,28,25,6,6,21,29,27,9,24,20
17,1610612750,Minnesota Timberwolves,82,29,53,0.354,48.5,37.7,81.3,0.464,...,29,8,23,14,19,17,18,7,15,24
22,1610612755,Philadelphia 76ers,82,10,72,0.122,48.3,36.2,84.0,0.431,...,30,20,29,10,3,28,25,25,29,30


In [14]:
sorted_pts.columns

Index(['TEAM_ID', 'TEAM_NAME', 'GP', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA',
       'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB',
       'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS',
       'PLUS_MINUS', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK',
       'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK',
       'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK',
       'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK',
       'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK'],
      dtype='object')

In [15]:
team_name_to_abbr = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BKN",
    "Charlotte Hornets": "CHO",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "LA Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP",
    "New York Knicks": "NYK",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHX",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTA",
    "Washington Wizards": "WAS"
}

for season, df in season_data_frames.items():
    if 'TEAM_NAME' not in df.columns:
        print(f"'TEAM_NAME' column missing in DataFrame for season: {season}")
    else:
        # Map team names to abbreviations
        df['TEAM_ABBREVIATION'] = df['TEAM_NAME'].map(team_name_to_abbr)
        df = df.rename(columns = {"TEAM_ABBREVIATION": "Team"}).drop(columns = ["TEAM_ID", "MIN", "GP", "W", "L"])
        
        # Update the dictionary with the modified DataFrame
        season_data_frames[season] = df

# Verify the mapping
df_2017_2018 = season_data_frames['2017-18']
df_2017_2018.head()





,TEAM_NAME,W_PCT,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,Team
0,Atlanta Hawks,0.293,38.2,85.5,0.446,11.2,31.0,0.360,15.8,20.2,...,8,27,12,23,28,14,14,25,26,ATL
1,Boston Celtics,0.671,38.3,85.1,0.450,11.5,30.4,0.377,16.0,20.7,...,20,15,23,18,6,17,22,20,6,BOS
2,Brooklyn Nets,0.341,38.2,86.8,0.441,12.7,35.7,0.356,17.4,22.6,...,9,25,30,16,27,23,19,14,24,BKN
3,Charlotte Hornets,0.439,39.0,86.7,0.450,10.0,27.2,0.369,20.2,27.0,...,24,3,28,18,20,2,1,10,17,CHO
4,Chicago Bulls,0.329,38.7,88.8,0.435,11.0,31.1,0.355,14.6,19.2,...,10,13,18,30,24,9,29,26,29,CHI


In [16]:
playoff_results = pd.read_csv('data/playoff_results.csv') 
playoff_results.tail()

,Yr,Lg,Series,Unnamed: 3,Unnamed: 4,Team,W,Unnamed: 7,Team.1,W.1,Unnamed: 10,Favorite,Underdog
130,2016,NBA,Western Conf Semifinals,May 1 - May 11 2016,NaN,Golden State Warriors (1),4,NaN,Portland Trail Blazers (5),1,NaN,GSW (-1600),POR (+1050)
131,2016,NBA,Western Conf Semifinals,Apr 30 - May 12 2016,NaN,Oklahoma City Thunder (3),4,NaN,San Antonio Spurs (2),2,NaN,SAS (-270),OKC (+230)
132,2016,NBA,Eastern Conf Finals,May 17 - May 27 2016,NaN,Cleveland Cavaliers (1),4,NaN,Toronto Raptors (2),2,NaN,CLE (-1350),TOR (+885)
133,2016,NBA,Western Conf Finals,May 16 - May 30 2016,NaN,Golden State Warriors (1),4,NaN,Oklahoma City Thunder (3),3,NaN,GSW (-430),OKC (+345)
134,2016,NBA,Finals,Jun 2 - Jun 19 2016,NaN,Cleveland Cavaliers (1),4,NaN,Golden State Warriors (1),3,NaN,GSW (-220),CLE (+180)


In [17]:
relabeled_dict = {
    "Team": "Team1",
    "Team.1": "Team2",
    "W": "Team1wins",
    "W.1": "Team2wins"
}

playoff_results = playoff_results.rename(columns=relabeled_dict)
playoff_results.head()

,Yr,Lg,Series,Unnamed: 3,Unnamed: 4,Team1,Team1wins,Unnamed: 7,Team2,Team2wins,Unnamed: 10,Favorite,Underdog
0,2024,NBA,Eastern Conf First Round,Apr 21 - May 1 2024,NaN,Boston Celtics (1),4,NaN,Miami Heat (8),1,NaN,BOS (-10000),MIA (+2000)
1,2024,NBA,Eastern Conf First Round,Apr 20 - May 5 2024,NaN,Cleveland Cavaliers (4),4,NaN,Orlando Magic (5),3,NaN,CLE (-210),ORL (+180)
2,2024,NBA,Eastern Conf First Round,Apr 21 - May 2 2024,NaN,Indiana Pacers (6),4,NaN,Milwaukee Bucks (3),2,NaN,IND (-125),MIL (+105)
3,2024,NBA,Eastern Conf First Round,Apr 20 - May 2 2024,NaN,New York Knicks (2),4,NaN,Philadelphia 76ers (7),2,NaN,NYK (-140),PHI (+120)
4,2024,NBA,Western Conf First Round,Apr 21 - Apr 29 2024,NaN,Oklahoma City Thunder (1),4,NaN,New Orleans Pelicans (8),0,NaN,OKC (-700),NOP (+500)


In [18]:
def clean_and_extract_team_id(team_name):
    # Remove seed numbers and parentheses
    cleaned_name = re.sub(r'\s\(\d+\)', '', team_name)
    # Extract first three letters and convert to uppercase
    team_id = cleaned_name[:3].upper()
    return team_id



playoff_results['Team1'] = playoff_results['Team1'].apply(clean_and_extract_team_id)
playoff_results['Team2'] = playoff_results['Team2'].apply(clean_and_extract_team_id) 
playoff_results.tail(15)

,Yr,Lg,Series,Unnamed: 3,Unnamed: 4,Team1,Team1wins,Unnamed: 7,Team2,Team2wins,Unnamed: 10,Favorite,Underdog
120,2016,NBA,Eastern Conf First Round,Apr 17 - Apr 24 2016,NaN,CLE,4,NaN,DET,0,NaN,CLE (-1700),DET (+1100)
121,2016,NBA,Eastern Conf First Round,Apr 16 - May 1 2016,NaN,TOR,4,NaN,IND,3,NaN,TOR (-400),IND (+325)
122,2016,NBA,Eastern Conf First Round,Apr 17 - May 1 2016,NaN,MIA,4,NaN,CHA,3,NaN,MIA (-170),CHO (+150)
123,2016,NBA,Eastern Conf First Round,Apr 16 - Apr 28 2016,NaN,ATL,4,NaN,BOS,2,NaN,ATL (-165),BOS (+145)
124,2016,NBA,Western Conf First Round,Apr 16 - Apr 27 2016,NaN,GOL,4,NaN,HOU,1,NaN,GSW (-10000),HOU (+4000)
125,2016,NBA,Western Conf First Round,Apr 17 - Apr 24 2016,NaN,SAN,4,NaN,MEM,0,NaN,SAS (-17000),MEM (+6625)
126,2016,NBA,Western Conf First Round,Apr 16 - Apr 25 2016,NaN,OKL,4,NaN,DAL,1,NaN,OKC (-3500),DAL (+1750)
127,2016,NBA,Western Conf First Round,Apr 17 - Apr 29 2016,NaN,POR,4,NaN,LOS,2,NaN,LAC (-525),POR (+415)
128,2016,NBA,Eastern Conf Semifinals,May 2 - May 8 2016,NaN,CLE,4,NaN,ATL,0,NaN,CLE (-500),ATL (+400)
129,2016,NBA,Eastern Conf Semifinals,May 3 - May 15 2016,NaN,TOR,4,NaN,MIA,3,NaN,TOR (-145),MIA (+125)


In [19]:
# Define the mapping of teams
team_replacements = {
    'LOS': 'LAC',
    'CHA': 'CHO',
    'GOL': 'GSW',
    'BRO': 'BKN',
    'OKL': 'OKC',
    'NEW': 'NYK',
    'PHO': 'PHX'
}

# Replace values in 'Team1' and 'Team2' columns
playoff_results['Team1'] = playoff_results['Team1'].replace(team_replacements)
playoff_results['Team2'] = playoff_results['Team2'].replace(team_replacements)

# Print the unique values in 'Team1' to verify the changes
print(playoff_results['Team1'].unique())


['BOS' 'CLE' 'IND' 'NYK' 'OKC' 'DAL' 'MIN' 'DEN' 'MIA' 'PHI' 'PHX' 'GSW'
 'LAC' 'MIL' 'MEM' 'BKN' 'UTA' 'ATL' 'TOR' 'HOU' 'POR' 'WAS' 'SAN']


In [20]:
playoff_results["Year"] = playoff_results["Unnamed: 3"].str.extract(r"(\d{4})").astype(int)

# Step 2: Reshape the data to combine Team1 and Team2 information
team1_data = playoff_results[["Year", "Team1", "Team1wins"]].rename(columns={"Team1": "Team", "Team1wins": "Wins"})
team2_data = playoff_results[["Year", "Team2", "Team2wins"]].rename(columns={"Team2": "Team", "Team2wins": "Wins"})

# Combine both datasets
all_teams = pd.concat([team1_data, team2_data])

# Step 3: Group by year and team to calculate total wins
team_wins_by_year = all_teams.groupby(["Year", "Team"], as_index=False)["Wins"].sum()
team_wins_by_year.sample(20)

,Year,Team,Wins
126,2024,CLE,5
125,2024,BOS,16
129,2024,IND,8
27,2017,POR,0
3,2016,CLE,16
24,2017,MEM,2
113,2023,CLE,1
85,2021,MEM,1
101,2022,MEM,6
127,2024,DAL,13


In [60]:
# Create an empty dictionary to store the merged data for each season
merged_season_data = {}

available_seasons = ['2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24']
# Iterate over the seasons and merge playoff wins with team stats
for season in available_seasons:
    # Extract the year (e.g., "2021" from "2021-22")
    season_year = int(season.split('-')[1]) + 2000
    # Filter playoff wins for the current season from team_wins_by_year
    playoff_wins_season = team_wins_by_year[team_wins_by_year['Year'] == int(season_year)]
    
    if not playoff_wins_season.empty:
        # Get the team stats for the current season
        df = season_data_frames[season]

        # Merge playoff wins with team stats
        df_merged = pd.merge(df, playoff_wins_season[['Team', 'Wins']], how='left', left_on='Team', right_on='Team')

        # Store the merged data in the dictionary
        merged_season_data[season] = df_merged

        # Display the merged data for the season
        print(f"Updated Merged Data for {season} season:")
        print(df_merged.head())  # Print the top rows of the merged data to verify
    else:
        print(f"No playoff win data found for season {season}")



Updated Merged Data for 2016-17 season:
           TEAM_NAME  W_PCT   FGM   FGA  FG_PCT  FG3M  FG3A  FG3_PCT   FTM  \
0      Atlanta Hawks  0.524  38.1  84.4   0.451   8.9  26.1    0.341  18.1   
1     Boston Celtics  0.646  38.6  85.1   0.454  12.0  33.4    0.359  18.7   
2      Brooklyn Nets  0.244  37.8  85.2   0.444  10.7  31.6    0.338  19.4   
3  Charlotte Hornets  0.439  37.7  85.4   0.442  10.0  28.6    0.351  19.4   
4      Chicago Bulls  0.500  38.6  87.1   0.444   7.6  22.3    0.340  18.0   

    FTA  ...  TOV_RANK  STL_RANK  BLK_RANK  BLKA_RANK  PF_RANK  PFD_RANK  \
0  24.9  ...        28         7        14         22        6         3   
1  23.2  ...         8        18        23         23       21        11   
2  24.6  ...        29        21        18         30       25         9   
3  23.8  ...         1        27        17         28        1        15   
4  22.5  ...        12        15        16         12        2        25   

   PTS_RANK  PLUS_MINUS_RANK  Team

In [26]:
# Initialize an empty dictionary to store the correlation tables for each season
season_correlations = {}

# Loop through each season's data in the season_data_frames
for season, season_data in season_data_frames.items():
    # Merge the season data with the playoff wins for the corresponding year
    season_year = int(season.split('-')[0]) + 1  # Extract the year (e.g., '2016-17' -> 2017)
    playoff_wins_season = team_wins_by_year[team_wins_by_year['Year'] == season_year]
    df_merged = pd.merge(season_data, playoff_wins_season[['Team', 'Wins']], how='left', left_on='Team', right_on='Team')
    
    # Select only numeric columns for correlation computation
    df_numeric = df_merged.select_dtypes(include=[float, int])

    # Calculate the correlation matrix for the numeric columns
    correlation_matrix = df_numeric.corr()

    # Get the correlation of each feature with Playoff_Wins (assuming the column is named 'Wins')
    playoff_wins_correlation = correlation_matrix['Wins'].sort_values(ascending=False)

    # Store the correlation table for the season
    season_correlations[season] = playoff_wins_correlation

# Display the correlations for all seasons
# Loop through the dictionary and drop the "Wins" entry from each correlation table (Series)
for season, correlation_table in season_correlations.items():
    correlation_table = correlation_table.drop('Wins')  # Drop the 'Wins' entry
    season_correlations[season] = correlation_table
    print(f"Correlation Table for {season}:")
    print(correlation_table)
    print("\n")



Correlation Table for 2015-16:
W_PCT              0.836893
PLUS_MINUS         0.824299
FG_PCT             0.717308
FG3_PCT            0.624916
FGM                0.601993
DREB               0.568975
PTS                0.557585
REB                0.458896
AST                0.376497
FG3M               0.349493
BLK                0.347824
FTA_RANK           0.347435
FTM_RANK           0.312541
STL_RANK           0.311053
PFD_RANK           0.305498
TOV_RANK           0.288676
TOV                0.262638
FT_PCT             0.206585
FG3A               0.171940
MIN_RANK           0.059238
OREB               0.045726
FGA                0.020382
FT_PCT_RANK        0.017062
OREB_RANK          0.016135
PF                -0.066070
FGA_RANK          -0.078287
FG3A_RANK         -0.121332
PF_RANK           -0.124695
FTM               -0.178282
FG3M_RANK         -0.203622
BLKA              -0.225720
FTA               -0.298295
AST_RANK          -0.318484
PFD               -0.322317
STL              

In [58]:
for season, correlation_table in season_correlations.items():
  print(f"most important features for {season}:")
  print(correlation_table.head(7))
  print("\n")

most important features for 2015-16:
W_PCT         0.836893
PLUS_MINUS    0.824299
FG_PCT        0.717308
FG3_PCT       0.624916
FGM           0.601993
DREB          0.568975
PTS           0.557585
Name: Wins, dtype: float64


most important features for 2016-17:
W_PCT         0.839129
PLUS_MINUS    0.773503
FG3M          0.669983
PTS           0.668914
AST           0.661640
FGM           0.608016
FG3A          0.586860
Name: Wins, dtype: float64


most important features for 2017-18:
FG3_PCT      0.645146
MIN_RANK     0.615397
OREB_RANK    0.566101
FG3M         0.561717
PTS          0.523472
W_PCT        0.511614
FGA_RANK     0.496020
Name: Wins, dtype: float64


most important features for 2018-19:
W_PCT         0.847822
PLUS_MINUS    0.708557
PTS           0.661446
FT_PCT        0.633279
FGM           0.629078
FG_PCT        0.623721
BLK           0.489866
Name: Wins, dtype: float64


most important features for 2019-20:
FTM           0.564891
PFD           0.562837
FTA           0.

In [62]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


def preprocess_combined_data(combined_season_data, selected_features):
    """
    Prepares data for training by splitting into features and target and applying scaling.
    """
    combined_season_data = combined_season_data.fillna(0)  # Replace NaN with 0
    
    # Target variable
    y_combined = combined_season_data['Wins']  # Use 'Wins' column as the target
    X_combined = combined_season_data[selected_features]  # Use only selected features for training

    return X_combined, y_combined


def train_combined_model(combined_season_data, selected_features):
    """
    Trains a model using data from all seasons combined.
    """
    X_combined, y_combined = preprocess_combined_data(combined_season_data, selected_features)
    
    # Split into training and test sets
    X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(
        X_combined, y_combined, test_size=0.2, random_state=42
    )
    
    # Standardize features
    scaler_combined = StandardScaler()
    X_train_scaled_combined = scaler_combined.fit_transform(X_train_combined)
    X_test_scaled_combined = scaler_combined.transform(X_test_combined)
    
    # Train the model
    model_combined = LinearRegression()
    model_combined.fit(X_train_scaled_combined, y_train_combined)
    
    # Evaluate model performance
    y_pred_combined = model_combined.predict(X_test_scaled_combined)
    mse_combined = mean_squared_error(y_test_combined, y_pred_combined)
    print(f"Combined Model Performance (MSE): {mse_combined}")
    
    return model_combined, scaler_combined


# Step 1: Combine all season data
all_season_data = pd.concat([merged_season_data[season] for season in available_seasons], axis=0)

# Step 2: Select common features across seasons (manually chosen or based on importance rankings)
selected_features_combined = ['W_PCT', 'PLUS_MINUS', 'FG3M', 'PTS', 'FG_PCT', 'AST', 'FG3_PCT', ]

# Step 3: Train the combined model
print("Training combined model with data from all seasons...")
combined_model_v1, combined_scaler_v1 = train_combined_model(all_season_data, selected_features_combined)

# Step 4: Predict playoff wins for the 2024-25 season using the combined model
X_2024_25_combined = season_data_frames['2024-25'][selected_features_combined]
X_2024_25_scaled_combined = combined_scaler_v1.transform(X_2024_25_combined)  # Scale features

# Generate predictions
predicted_wins_2024_25_combined = combined_model_v1.predict(X_2024_25_scaled_combined)
predicted_wins_2024_25_combined = [max(0, win) for win in predicted_wins_2024_25_combined]  # Ensure no negative values

# Combine predictions with team names
predictions_combined = pd.DataFrame({
    'Team': season_data_frames['2024-25']['TEAM_NAME'],
    'Predicted Playoff Wins': predicted_wins_2024_25_combined
})

# Display predictions
print(predictions_combined)


Training combined model with data from all seasons...
Combined Model Performance (MSE): 10.03820850973621
                      Team  Predicted Playoff Wins
0            Atlanta Hawks                2.681832
1           Boston Celtics                8.795857
2            Brooklyn Nets                1.317367
3        Charlotte Hornets                0.000000
4            Chicago Bulls                4.204341
5      Cleveland Cavaliers               11.379968
6         Dallas Mavericks                4.639523
7           Denver Nuggets                4.437612
8          Detroit Pistons                4.028692
9    Golden State Warriors                4.035427
10         Houston Rockets                4.765214
11          Indiana Pacers                4.294883
12             LA Clippers                4.137058
13      Los Angeles Lakers                3.821101
14       Memphis Grizzlies                5.422910
15              Miami Heat                4.593476
16         Milwaukee Bucks 

In [63]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict

def train_and_evaluate_rf_model(all_season_data, selected_features, n_estimators=100, max_depth=None):
    X_combined, y_combined = preprocess_combined_data(all_season_data, selected_features)
    
    scaler_rf = StandardScaler() # Standardize features 
    X_scaled = scaler_rf.fit_transform(X_combined)

    # Training random forest model
    rf_model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    rf_model.fit(X_scaled, y_combined)
    
    # Cross-validation predictions for combined data
    cross_val_predictions = cross_val_predict(rf_model, X_scaled, y_combined, cv=5)
    avg_mse_rf = ((cross_val_predictions - y_combined) ** 2).mean()  # Mean Squared Error
    print(f"Combined Random Forest Model Performance (Avg MSE): {avg_mse_rf}")
    
    # getting predictions for each season
    season_predictions = {}
    for season, data_frame in merged_season_data.items():
        X_season = data_frame[selected_features]
        y_season = data_frame['Wins']
        X_season_scaled = scaler_rf.transform(X_season)
        season_predictions[season] = {
            'Team': data_frame['TEAM_NAME'],
            'Actual Wins': y_season,
            'Predicted Wins': rf_model.predict(X_season_scaled)
        }
    
    return rf_model, scaler_rf, season_predictions

# Train and evaluate the Random Forest model
print("Training and evaluating combined Random Forest model with data from all seasons...")
rf_model_combined, rf_scaler_combined, all_season_predictions = train_and_evaluate_rf_model(
    all_season_data, selected_features_combined, n_estimators=500, max_depth=10
)

# Display predictions for each season
for season, results in all_season_predictions.items():
    predictions_df = pd.DataFrame(results)
    print(f"\nPredictions for {season}:")
    print(predictions_df)


Training and evaluating combined Random Forest model with data from all seasons...
Combined Random Forest Model Performance (Avg MSE): 9.783537167912474

Predictions for 2016-17:
                      Team  Actual Wins  Predicted Wins
0            Atlanta Hawks          2.0        2.234677
1           Boston Celtics          9.0        8.576510
2            Brooklyn Nets          NaN        0.000087
3        Charlotte Hornets          NaN        0.114308
4            Chicago Bulls          2.0        1.505500
5      Cleveland Cavaliers         13.0       10.864613
6         Dallas Mavericks          NaN        0.008000
7           Denver Nuggets          NaN        0.026276
8          Detroit Pistons          NaN        0.396000
9    Golden State Warriors         16.0       14.204000
10         Houston Rockets          6.0        7.378106
11          Indiana Pacers          0.0        0.614532
12             LA Clippers          3.0        3.383383
13      Los Angeles Lakers          N

In [74]:
##calculating rmse for each season in my model

def calculate_rmse_per_season(all_season_predictions):
    season_rmse = {}
    
    for season, results in all_season_predictions.items():
        actual_wins = results['Actual Wins']
        predicted_wins = results['Predicted Wins']

        actual_wins = actual_wins.fillna(0)  # replace NaN values w 0 in actual wins
        

        mse = mean_squared_error(actual_wins, predicted_wins)  # Calculate rmse for the season
        rmse = np.sqrt(mse)
        
        season_rmse[season] = rmse
    
    return season_rmse


season_rmse = calculate_rmse_per_season(all_season_predictions)

# Print RMSE for each season
for season, rmse in season_rmse.items():
    print(f"RMSE for {season}: {rmse}")

RMSE for 2016-17: 1.042650713560256
RMSE for 2017-18: 1.0318190034853882
RMSE for 2018-19: 0.8359862606702951
RMSE for 2019-20: 1.5796865695820332
RMSE for 2020-21: 1.0566795091929322
RMSE for 2021-22: 0.9516032727799066
RMSE for 2022-23: 1.3264949279741693
RMSE for 2023-24: 1.200110483066244


In [73]:
#average rmse of the model

avg_rmse = np.mean(list(season_rmse.values()))
print("Average RMSE across all seasons:", avg_rmse)

Average RMSE across all seasons: 1.128128842538903


In [71]:
##predictions for the 2025 playoffs!

current_year = season_data_frames['2024-25']
X_current = current_year[selected_features_combined]
X_current_scaled = rf_scaler_combined.transform(X_current)

current_year_predictions = rf_model_combined.predict(X_current_scaled)
current_year['Predicted_Wins_2025'] = current_year_predictions
print("Predictions for the current year (2025):")
print(current_year[['Team', 'Predicted_Wins_2025']])


Predictions for the current year (2025):
   Team  Predicted_Wins_2025
0   ATL             0.627293
1   BOS             8.898772
2   BKN             0.012000
3   CHO             0.128087
4   CHI             0.673662
5   CLE            12.476000
6   DAL             2.672296
7   DEN             9.317659
8   DET             0.517335
9   GSW             0.471340
10  HOU             9.287133
11  IND             1.734767
12  LAC             4.447616
13  LAL             2.238378
14  MEM            11.798833
15  MIA             2.566448
16  MIL             2.252293
17  MIN             1.277275
18  NOP             0.034087
19  NYK            11.464681
20  OKC             8.607735
21  ORL             3.531579
22  PHI             0.040087
23  PHX             0.393439
24  POR             0.000087
25  SAC             0.999599
26  SAS             0.131147
27  TOR             0.000000
28  UTA             0.000000
29  WAS             0.030087
